In [69]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
#import src
#import Cleanup
#from millify import millify
import numpy as np
import pandas as pd
from pathlib import Path



In [70]:
class Transformation():
    def trial_func(self):
        '''
        Function to verofy that the class works
        '''
        print(f"transform:Class called")  # duplicate values have been removed")

        return True

    def convertFeetToMetre(self, df):
        '''
        Convert the input variables which exist in Feet to Metres or Feet/Second to Metre/Second and so on. 
        Input: 
            df
        Ouptut: 
            df
        '''
        df['v_length'] = df['v_length']*0.3048
        df['Space_Headway'] = df['Space_Headway']*0.3048
        df['v_Vel'] = df['v_Vel']*0.3048
        df['v_Acc'] = df['v_Acc']*0.3048
        
        return df

    def dropNotRequiredColumns(self, df):
        '''
        Drop the columns that are not required for our Model.
        Input: 
            df
        Ouptut: 
            df
        '''
        df = df.drop(columns=['Movement', 'Direction', 'Section_ID', 'Int_ID',
                     'D_Zone', 'O_Zone', 'Following', 'v_Width', 'Total_Frames'])
        return df

    def createColumnPlaceholders(self, df):
        '''
        Create PLaceholder for columns that will be generated and populated to use in the Model 
        Input: 
            df
        Ouptut: 
            df
        '''
        df['Preceding_Vehicle_Class'] = np.NaN
        df['Rear_to_Front_Space_Headway'] = np.NaN
        df['Front_To_Rear_Time_Headway'] = np.NaN
        df['Velocity Difference_Following-Preceding'] = np.NaN
        df['Acceleration Difference_Following-Preceding'] = np.NaN
        df['L-F_Pair'] = df['Preceding'].astype(str) + '-' + df['Vehicle_ID'].astype(str)
        df["v_Class_Name"] = df["v_Class"].map({1:"Motorcycle", 2: "Car", 3: "Heavy Vehicle"})
        df['Relative_Time']= df['Global_Time'] - df['Global_Time'].min() + 1
        return df

    def bifurcateHighways(self, df):
        '''
        filter out US-101 and I-80 into separate HIghway dataframes as same Vehicle IDs exist in both Hughways.
        Input: 
            df
        Ouptut: 
            df
        '''

        filtered_U = df[((df['Location'] == 'us-101'))]

        filtered_I = df[(df['Location'] == 'i-80')]

        return filtered_U, filtered_I

    def classVehicleSets(self, df):
        '''
        Create the set for the vehicle classes, Motorcycle, Car and Heavy Vehicle
        Input: 
            df
        Output:
            Vehicle ID set in sequence: 1. Motorcycle , 2. Car, 3. Heavy Vehicle
        '''

        filtered_vClass = df[['Vehicle_ID', 'v_Class',
                              'v_length']].drop_duplicates().sort_values('v_Class')
        v_Class_M = set(
            filtered_vClass[(filtered_vClass['v_Class'] == 1)]['Vehicle_ID'])
        v_Class_C = set(
            filtered_vClass[(filtered_vClass['v_Class'] == 2)]['Vehicle_ID'])
        v_Class_HV = set(
            filtered_vClass[(filtered_vClass['v_Class'] == 3)]['Vehicle_ID'])

        return v_Class_M, v_Class_C, v_Class_HV

    def precedingVehicleClass(self, df, v_Class_M, v_Class_C, v_Class_HV):
        '''
        Find and populate the preceding Vehicle Class Name into Preceding_Vehicle_Class column, and populate the Vehicle_combination as well. 
        Input: 
            df,the three deciding Class sets of Motorcycle and Heavy Vehicle. 
        Ouptut: 
            df
        '''
        result = []

        for i in df.index:
            if df['Preceding'][i] in v_Class_C:
                result.append('Car')
            elif df['Preceding'][i] in v_Class_HV:
                result.append('Heavy Vehicle')
            elif df['Preceding'][i] in v_Class_M:
                result.append('Motorcycle')
            else:
                result.append('Free Flow')
        df['Preceding_Vehicle_Class'] = result
        df['Vehicle_combination'] = df['Preceding_Vehicle_Class'] + \
            '-' + df['v_Class_Name']
        return df

    def precedingVehicleLength(self, df):
        '''
        Find and populate the preceding Vehicle Length Name into preceding_vehicle_length column. If there is no Preceding vehicle, it will be populated with 0 in case of Vehicle ID 0
        Input: 
            df
        Ouptut: 
            df
        '''
        vehicle_lengths = df[['Vehicle_ID', 'v_length']]
        # print(f"{vehicle_lengths.duplicated().sum()} duplicate values have been removed")
        vehicle_lengths.drop_duplicates(inplace=True)
        # print(vehicle_lengths.shape)
        x = vehicle_lengths.groupby(['Vehicle_ID']).mean()
        dict = x.to_dict()['v_length']
        df["preceding_vehicle_length"] = df["Preceding"].map(dict)
        df["preceding_vehicle_length"] = df["preceding_vehicle_length"].fillna(
            0)
        return df

    def frontToFrontBumperDetailsChangedToRearToFrontBumperDetails(self, df):
        '''
        Change the details from the Front to Front Bumper to Rear of Lead to Front Bumper of Subject Vehicle.
            1. Space Headway
            2. Time Headway
        Input: 
            df
        Ouptut: 
            df
        '''
        df['Rear_to_Front_Space_Headway'] = df['Space_Headway'] - \
            df['preceding_vehicle_length']
        df['Front_To_Rear_Time_Headway'] = df['Rear_to_Front_Space_Headway'] / df['v_Vel']
        return df

    def mapPreviousVehicleDetails(self, df):
        '''
        Update Preceding Vehicle Details for the below columns
            1. Previous Vehicle Acceleration.
            2. Previous Vehicle Velocity.
            3. Previous Vehicle Lane Change details
            4. Populate any missing details for Vehicle ID 0 with either 0 or False. 
        Input: 
            df
        Ouptut: 
            df
        '''
        lane_verify = df[['Vehicle_ID', 'Lane_ID']]
        # print(f"{lane_verify.duplicated().sum()} duplicate values have been removed")
        lane_verify.drop_duplicates(inplace=True)
        lane_verify = lane_verify.groupby(
            ['Vehicle_ID'], as_index=False).count()
        # print(lane_verify.shape)
        lane_change_vehicles = set(
            lane_verify[lane_verify["Lane_ID"] > 1]['Vehicle_ID'])
        df['lane_changes'] = df['Vehicle_ID'].isin(
            lane_change_vehicles)
        print(
            f"{df['Location']}::{df[(df['lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars dont change lanes")
        print(
            f"{df['Location']}::{df[(df['lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
        right_df = df[['Preceding', 'Relative_Time',
                       'v_Vel', 'v_Acc', 'lane_changes']]
        right_df.rename(columns={'Preceding': 'Prec_Vehicle_ID', 'v_Vel': 'previous_Vehicle_Velocity',
                        'v_Acc': 'previous_Vehicle_Acceleration', "lane_changes": "previous_car_lane_changes"}, inplace=True)
        df['Prec_Vehicle_ID'] = df['Vehicle_ID']
        df = df.merge(right=right_df, how='left', on=(
            'Prec_Vehicle_ID', 'Relative_Time'))
        df['previous_Vehicle_Velocity'] = df['previous_Vehicle_Velocity'].fillna(
            0)
        df['previous_Vehicle_Acceleration'] = df['previous_Vehicle_Acceleration'].fillna(
            0)
        df['previous_car_lane_changes'] = df['previous_car_lane_changes'].fillna(
            False)
        return df

    def mapPairs(self, df):
        '''
        Map the Pairs for the Preceding and lead vehicle. 
        Input: 
            df
        Ouptut: 
            df
        '''
        df['Velocity Difference_Following-Preceding'] = df['v_Vel'] - \
            df['previous_Vehicle_Velocity']
        df['Acceleration Difference_Following-Preceding'] = df['v_Acc'] - \
            df['previous_Vehicle_Acceleration']
        df = df.sort_values(by=['Relative_Time'],
                            ascending=True, ignore_index=True)
        df['pair_Time_Duration'] = (df.groupby(
            ['L-F_Pair'], as_index=False).cumcount()*0.1)
        x = (df[['L-F_Pair', 'pair_Time_Duration']].groupby(['L-F_Pair'],
                                                            as_index=False).max(['pair_Time_Duration']))
        dict_lenght = dict(zip(x['L-F_Pair'], x['pair_Time_Duration']))
        df["total_pair_duration"] = df["L-F_Pair"].map(dict_lenght)
        print(df["total_pair_duration"].dtype)
        return df

 

In [71]:
class FileProcessing():
    p = Path().cwd()
    stringpath = str(p)[0:str(p).rfind('\\')] + '\\data'

#    def __init__(self):
#        self.initialize()

    def readInput(self, fileName):
        '''
        Read the input file into a dataframe. 
        Input: File name for the file present in Data folder. 
        Output: Dataframe name. 
        '''
        print(f"original File path: {self.p}")
        print(f"Data File path: { self.stringpath}")
        ngsimfile = self.stringpath + '/' + fileName + '.csv'
        df = pd.read_csv(ngsimfile, low_memory=False)
        return df

    def exportFile(self, df, fileName):
        '''
        Export the working Data frame into csv file of the mentioned name.  
        Input: 
            df
        Ouptut: 
            df
        '''
    
        ngsimfilteredfile = self.stringpath + '\\' + fileName + '.csv'
        df.to_csv(ngsimfilteredfile, index=False)
        return True
   
    def mergeFiles(self, df1, df2):
        '''
        Merge the I-80 and US-101 Highway dataframe.  
        Input: 
            df
        Ouptut: 
            df
        '''
        df = pd.concat([df1, df2])
        print(df1.shape[0])
        print(df2.shape[0])
        print(df.shape[0])
        
        return df



In [72]:
class Cleanup():
    # def __init__(self):
    #    self.initialize()

    def trial_func(self):
        '''
        Function Removes Duplicates from the Dataframe
        '''
        print(f"Class called")  # duplicate values have been removed")
        return True

    def removeDups(self,df):
        '''
        Remove Duplicates.
        Input: 
            df
        Output:
            df
        '''
        print(f"{df.duplicated().sum()} duplicate values have been removed")
        df.drop_duplicates(inplace=True)
        return df
        
    def filterRecordsForModel(self, df):
        '''
        Map the Pairs for the Preceding and lead vehicle. 
        Input: 
            df
        Ouptut: 
            df
        '''
        v_Class_verify= df[['Vehicle_ID','v_Class']]
        v_Class_verify.drop_duplicates(inplace=True)
        v_Class_verify=v_Class_verify.groupby(['Vehicle_ID'],as_index=False).count()
        remove_bad_data_vehicles=set(v_Class_verify[v_Class_verify["v_Class"]>1]['Vehicle_ID'])
        bad_v_Class_length=df[(df['Vehicle_ID'].isin(remove_bad_data_vehicles))]

        total_duration_less_than_minute = df[(
            df['total_pair_duration'] >= 6)]
        total_duration_less_than_minute.index
        both_lane_change = df[(df['previous_car_lane_changes'] == True) & (df['lane_changes'] == True) & (
            (df['pair_Time_Duration'] <= 5) | (df['pair_Time_Duration'] >= (df['total_pair_duration'] - 5)))]
        lead_change = df[(df['previous_car_lane_changes'] == True) & (
            df['lane_changes'] == False) & ((df['pair_Time_Duration'] <= 5))]
        subject_change = df[(df['previous_car_lane_changes'] == False) & (
            df['lane_changes'] == True) & (df['pair_Time_Duration'] >= (df['total_pair_duration'] - 5))]
        total_duration_less_than_minute = df[(
            df['total_pair_duration'] < 60)]
        before = df.shape[0]
        print(f"dataset before Row Removal{df.shape}")
        print(f"{both_lane_change.shape} Lead and Subject both change lane, so first and last 5 seconds of trajectory removed")
        print(f"{lead_change.shape} Lead Vehicle changes Lane, so first 5 seconds of car following Removed. ")
        print(f"{bad_v_Class_length} have multiple lengths and classes for same Vehicle ID")
        print(f"{subject_change.shape} subject vehicles change lanes so last 5 seconds of vehicle trajectory removed")
        remove = pd.concat([both_lane_change, lead_change,
                           subject_change, total_duration_less_than_minute,bad_v_Class_length])

        df.drop(labels=remove.index, inplace=True)
        after = df.shape[0]
        removed_row_count =  before - after
        print(f"{removed_row_count} rows removed for the first and last 5 seconds of the cars that changed lanes")
        df['Vehicle_ID'].isna().sum()

        return df



In [73]:
#define the objects to be called later. 
cleanup=Cleanup()
transform=Transformation()
fileProcessing=FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [74]:
filename='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.readInput(filename)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


Remove any Duplicate records from the original File. 

In [75]:
cleanup.removeDups(ngsim)

704000 duplicate values have been removed


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101
6,1890,9157,628,1118849672700,53.514,817.521,6451655.238,1872800.663,24.0,8.5,...,NaN,NaN,NaN,NaN,NaN,1882,1897,102.65,2.27,us-101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11850521,1355,5622,938,1113438127100,18.275,1464.336,6042648.149,2134520.896,12.3,6.8,...,NaN,NaN,NaN,NaN,NaN,1351,1361,26.91,15.92,i-80
11850522,1474,5688,401,1113438133700,5.509,560.412,6042767.840,2133624.549,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1468,1480,74.49,1.73,i-80
11850523,398,2368,654,1113437801700,18.903,635.841,6042771.741,2133701.076,15.2,8.5,...,NaN,NaN,NaN,NaN,NaN,385,406,90.95,3.77,i-80
11850524,599,875,577,1113436854400,77.094,621.358,6042831.282,2133693.854,15.3,6.4,...,NaN,NaN,NaN,NaN,NaN,0,611,0.00,0.00,i-80


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [76]:
ngsim=transform.convertFeetToMetre(ngsim)

In [77]:
ngsim.shape

(11146526, 25)

In [78]:
ngsim=transform.dropNotRequiredColumns(ngsim)

Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [79]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim=transform.createColumnPlaceholders(ngsim)

Split data into US-101 and I-80 specific dataframes.

In [80]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcateHighways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [81]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.classVehicleSets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.classVehicleSets(filtered_ngsim_I)


Add the Preceding Vehicle Class to the new previous class column

In [82]:
filtered_ngsim_U=transform.precedingVehicleClass(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.precedingVehicleClass(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [83]:
filtered_ngsim_U=transform.precedingVehicleLength(filtered_ngsim_U)
filtered_ngsim_I=transform.precedingVehicleLength(filtered_ngsim_I)

Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [84]:
filtered_ngsim_U= transform.frontToFrontBumperDetailsChangedToRearToFrontBumperDetails(filtered_ngsim_U)
filtered_ngsim_I= transform.frontToFrontBumperDetailsChangedToRearToFrontBumperDetails(filtered_ngsim_I)

Vehicle Combination of the Preceding and the Following Vehicle. 

In [85]:
filtered_ngsim_U=transform.mapPreviousVehicleDetails(filtered_ngsim_U)
filtered_ngsim_I=transform.mapPreviousVehicleDetails(filtered_ngsim_I)

0          us-101
3          us-101
4          us-101
6          us-101
7          us-101
            ...  
8704795    us-101
8704797    us-101
8704798    us-101
8704799    us-101
8704800    us-101
Name: Location, Length: 4098933, dtype: object::621 cars dont change lanes
0          us-101
3          us-101
4          us-101
6          us-101
7          us-101
            ...  
8704795    us-101
8704797    us-101
8704798    us-101
8704799    us-101
8704800    us-101
Name: Location, Length: 4098933, dtype: object::2226 cars Change lanes
2           i-80
10          i-80
12          i-80
13          i-80
14          i-80
            ... 
11850521    i-80
11850522    i-80
11850523    i-80
11850524    i-80
11850525    i-80
Name: Location, Length: 4566387, dtype: object::817 cars dont change lanes
2           i-80
10          i-80
12          i-80
13          i-80
14          i-80
            ... 
11850521    i-80
11850522    i-80
11850523    i-80
11850524    i-80
11850525    i-80
Name: Loc

In [86]:
filtered_ngsim_U=transform.mapPairs(filtered_ngsim_U)
filtered_ngsim_I=transform.mapPairs(filtered_ngsim_I)

filtered_ngsim_U=cleanup.filterRecordsForModel(filtered_ngsim_U)
filtered_ngsim_I=cleanup.filterRecordsForModel(filtered_ngsim_I)

float64
float64
dataset before Row Removal(4098933, 33)
(719729, 33) Lead and Subject both change lane, so first and last 5 seconds of trajectory removed
(36238, 33) Lead Vehicle changes Lane, so first 5 seconds of car following Removed. 
         Vehicle_ID  Frame_ID    Global_Time  Local_X   Local_Y     Global_X  \
60               14        32  1118846982100   49.079    49.741  6451122.979   
61               14        33  1118846982200   49.059    53.742  6451125.668   
68               14        34  1118846982300   49.037    57.741  6451128.357   
72               14        35  1118846982400   49.017    61.741  6451131.045   
80               14        36  1118846982500   48.997    65.742  6451133.734   
...             ...       ...            ...      ...       ...          ...   
4098193        1945      9828  1118849739800   54.446  2143.291  6452659.035   
4098206        1945      9829  1118849739900   54.443  2147.803  6452662.556   
4098228        1945      9830  1118849740


1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [87]:
filtered_ngsim= fileProcessing.mergeFiles(filtered_ngsim_U , filtered_ngsim_I)

fileProcessing.exportFile(filtered_ngsim,'Cleaned_NGSIM_Data')

1915446
1945651
3861097


True

In [88]:
print(f"{filtered_ngsim[(filtered_ngsim['previous_car_lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
print(f"{filtered_ngsim[(filtered_ngsim['previous_car_lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars DonT")

2387 cars Change lanes
2484 cars DonT


In [89]:
#filtered_ngsim[((filtered_ngsim['Vehicle_ID'] <= 100 )  ) & ((filtered_ngsim['Preceding'] <= 100 )  )].to_csv('classVERIFYFILE.CSV', index = False)

In [90]:
filtered_ngsim

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Relative_Time,Vehicle_combination,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,previous_Vehicle_Velocity,previous_Vehicle_Acceleration,previous_car_lane_changes,pair_Time_Duration,total_pair_duration
0,5,8,1118846979700,39.788,39.154,6451122.815,1873326.569,5.18160,2,12.1920,...,1117683960601,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.0,160.8
1,5,9,1118846979800,39.767,43.153,6451125.503,1873323.608,5.18160,2,12.1920,...,1117683960701,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.1,160.8
2,5,10,1118846979900,39.747,47.154,6451128.192,1873320.646,5.18160,2,12.1920,...,1117683960801,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.2,160.8
3,5,11,1118846980000,39.726,51.154,6451130.881,1873317.684,5.18160,2,12.1920,...,1117683960901,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.3,160.8
4,5,12,1118846980100,39.705,55.153,6451133.569,1873314.723,5.18160,2,12.1920,...,1117683961001,Free Flow-Car,0.00000,True,5,0.0,0.0,False,0.4,160.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566239,2485,11599,1113438724800,54.755,1595.253,6042663.224,2134655.863,4.20624,2,0.0762,...,1112275705701,Car-Car,4.52628,True,2485,0.0,0.0,False,122.9,128.4
4566241,2485,11600,1113438724900,54.755,1595.278,6042663.220,2134655.888,4.20624,2,0.0762,...,1112275705801,Car-Car,4.52628,True,2485,0.0,0.0,False,123.0,128.4
4566242,2485,11601,1113438725000,54.755,1595.303,6042663.216,2134655.912,4.20624,2,0.0762,...,1112275705901,Car-Car,4.52628,True,2485,0.0,0.0,False,123.1,128.4
4566245,2485,11602,1113438725100,54.756,1595.328,6042663.213,2134655.937,4.20624,2,0.0762,...,1112275706001,Car-Car,4.52628,True,2485,0.0,0.0,False,123.2,128.4


In [91]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )) ]


,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Relative_Time,Vehicle_combination,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,previous_Vehicle_Velocity,previous_Vehicle_Acceleration,previous_car_lane_changes,pair_Time_Duration,total_pair_duration
1113491,2964,8731,1118847852000,4.868,309.729,6451336.355,1873154.258,5.02920,2,2.618232,...,1117684832901,Car-Car,4.72440,False,2964,0.0000,0.0,False,29.7,89.6
2553891,2491,7916,1113437558500,17.161,598.884,6042774.601,2133664.163,4.66344,2,3.453384,...,1112274539401,Car-Car,4.51104,True,2491,3.5814,0.0,False,33.0,99.8
